In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
from splitbert.textsplit import text_segmentation
from splitbert.splitbert import SplitBertConcatEncoderModel
from splitbert.splitbert import conduct_input_ids_and_attention_masks
from splitbert.splitbert import make_masks
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import tqdm

2023-09-14 11:25:46.770941: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Data Preparation

In [2]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')
reply_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/avg_satisfaction_raw_0-999.csv', encoding='ISO-8859-1')

modes = [['seg', 'seg', 'snt']]

nlp = English()
nlp.add_pipe("sentencizer")

# satisfaction score (y)
satisfactions_float = list(reply_df['satisfy_composite'])
satisfactions = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)

reply_contents = list(reply_df['replyContent'])
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])


def get_sequences(contents, mode):
    sequences = []

    if mode == 'all':
        for content in contents:
            sequences.append([content])
    elif mode == 'seg':
        for content in contents:
            sentences = list(map(lambda x: str(x), list(nlp(content).sents)))
            sequences.append(text_segmentation(sentences))
    else:  # sentences
        for content in contents:
            sequences.append(list(map(lambda x: str(x), list(nlp(content).sents))))

    return sequences


for mode in modes:
    print(mode)
    post_sequences = get_sequences(post_contents, mode[0])
    comment_sequences = get_sequences(comment_bodies, mode[1])
    reply_sequences = get_sequences(reply_contents, mode[2])

    data = []
    max_post, max_comment, max_reply = 0, 0, 0
    i = 0
    for post, comment, reply, satisfaction, satisfaction_float in zip(post_sequences, comment_sequences,
                                                                          reply_sequences, satisfactions,
                                                                          satisfactions_float):
        if len(post) > max_post:
            max_post = len(post)
        if len(comment) > max_comment:
            max_comment = len(comment)
        if len(reply) > max_reply:
            max_reply = len(reply)

        data.append([i, post, comment, reply, satisfaction, satisfaction_float])
        i += 1

    print(max_post, max_comment, max_reply)
    max_count = max(max_post, max_comment, max_reply)
    print(max_count)

    columns = ['index', 'post_contents', 'comment_contents', 'reply_contents', 'label', 'score']
    df = pd.DataFrame(data, columns=columns)

    # data split (train & test sets)
    idx_train, idx_remain = train_test_split(df.index.values, test_size=0.20, random_state=42)
    idx_val, idx_test = train_test_split(idx_remain, test_size=0.50, random_state=42)

    train_df = df.iloc[idx_train]
    val_df = df.iloc[idx_val]
    test_df = df.iloc[idx_test]

    count_min_label = min(train_df['label'].value_counts())

    labels = [0, 1, 2]

    train_sample_df = pd.DataFrame([], columns=columns)

    for label in labels:
        tmp = train_df[train_df['label'] == label]
        tmp_sampled = tmp.sample(frac=1).iloc[:count_min_label]
        train_sample_df = pd.concat([train_sample_df, tmp_sampled])

    train_sample_df = train_sample_df.sample(frac=1)

['seg', 'seg', 'snt']
10 4 10
10


# Model Preparation

In [32]:
def normalize_tensor(data):
    return (data - torch.min(data)) / (torch.max(data) - torch.min(data))

In [3]:
def forward_func_ig(inputs):
    now = 0

    for embeddings, count in zip(inputs, sentence_counts):
        embeddings = embeddings.unsqueeze(0)
        embeddings = embeddings.swapaxes(0, 1)

        embeddings = model.pe(embeddings)

        src_mask, src_key_padding_mask = make_masks(model.max_len, count, device)

        encoder_output = model.encoder(embeddings, mask=src_mask, src_key_padding_mask=src_key_padding_mask)

        encoder_output = torch.mean(encoder_output[:count], dim=0)

        if now == 0:
            result_outputs = encoder_output
        else:
            result_outputs = torch.cat([result_outputs, encoder_output], dim=1)
        now += 1
    
    outputs = model.classifier1(result_outputs)
    logits = model.classifier2(outputs)
    return logits

In [24]:
def forward_func_ig2(inputs, sentence_counts, model):
    now = 0

    for encoder_output, count in zip(inputs, sentence_counts):
        encoder_output = encoder_output.swapaxes(0, 1)
        encoder_output = torch.mean(encoder_output[:count], dim=0)

        if now == 0:
            result_outputs = encoder_output
        else:
            result_outputs = torch.cat([result_outputs, encoder_output], dim=1)
        now += 1
        
    outputs = model.classifier1(result_outputs)
    logits = model.classifier2(outputs)
    return logits

ig = IntegratedGradients(forward_func_ig2)

In [21]:
# hidden_states == attention
def forward_func_ig3(inputs, sentence_counts, model):
    inputs = torch.stack(inputs, dim=0)
    inputs = torch.flatten(inputs)
    inputs = inputs.unsqueeze(0)
    inputs = model.attn_classifier1(inputs)
    inputs = model.attn_classifier2(inputs)
    logits = model.classifier2(inputs)
    
    return logits

ig = IntegratedGradients(forward_func_ig3)

In [53]:
# hidden_states = bert embeddings
def forward_func_ig4(inputs, sentence_counts, model):
    p_count, c_count = sentence_counts[0], sentence_counts[1]
    
    src_mask, src_key_padding_mask = make_masks(model.max_post_len, [p_count, c_count], model.device,
                                                model.max_post_len, model.max_comment_len, model.concat_mode)
    encoder_output = model.encoder(inputs, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
    
    now_attention = torch.empty(size=(p_count + c_count, 1, model.embedding_size), requires_grad=True).to(model.device)
    
    if model.attention_mode:
        attention = model.encoder_layer.self_attn(encoder_output, encoder_output, encoder_output,
                                                 attn_mask=src_mask, key_padding_mask=src_key_padding_mask)[0]
        
        if model.attention_mode == 'attention_residual':
            # add & norm
            residual = outputs_list[i]
            attention = model.dropout(attention) + residual
            attention = model.layer_norm(attention)

        # mul mask - diagonal masking
        attention = attention.swapaxes(0, 2)
        mask = torch.tensor([1] * (p_count + c_count) + [0] * (model.max_post_len + model.max_comment_len - (p_count + c_count))).to(model.device)
        attention = attention.mul(mask).swapaxes(0, 2)

        attention = torch.flatten(attention)
        attention = attention.unsqueeze(0)
        attention = model.attn_classifier1(attention)
        attention = model.attn_classifier2(attention)
        logits = model.classifier2(attention)
    
    return logits

ig = IntegratedGradients(forward_func_ig4)

In [5]:
def prepare_model(target, mode, epoch):
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device('cpu')

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    model_path = f'../predicting-satisfaction-using-graphs/splitbert/model/{mode}/epoch_{epoch}.model'
    model = SplitBertConcatEncoderModel(num_labels=len(labels), embedding_size=384, max_len=max_count, 
                                        max_post_len=max_post, max_comment_len=max_comment,
                                        device='cpu', target=target, concat_mode="concat_all",
                                       attention_mode="attention", output_attentions=True)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to('cpu')
    model.eval()

    for param in model.sbert.parameters():
        param.requires_grad = False
    
    for param in model.bert.parameters():
        param.requires_grad = False
        
    return device, model, tokenizer

In [6]:
device, pc_model, tokenizer = prepare_model('post_comment', 'seg_seg', 5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
device, t_model, tokenizer = prepare_model('triplet', 'seg_seg_snt', 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def construct_input_ref_pair(targets):
    input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list = [], [], [], []
    
    for contents in targets:
        result = tokenizer(contents, pad_to_max_length=True, truncation=True, max_length=256, return_tensors='pt')
        
        input_ids = result['input_ids']
        sentence_count_list.append(torch.tensor(len(input_ids)).unsqueeze(0))
        attention_masks = result['attention_mask']
        
        pad = (0, 0, 0, max_count-len(input_ids))
        input_ids = nn.functional.pad(input_ids, pad, "constant", 0)
        attention_masks = nn.functional.pad(attention_masks, pad, "constant", 0)
        ref_input_ids = torch.zeros_like(input_ids)

        input_ids_list.append(input_ids.unsqueeze(0))
        ref_input_ids_list.append(ref_input_ids.unsqueeze(0))
        attention_masks_list.append(attention_masks.unsqueeze(0))
    
    return input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list

In [14]:
def summarize_attributions(attribution):
    attributions = attribution.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [42]:
def splitbert_integrated_gradient_triplet(index, post, comment, reply, p_sentences, c_sentences, r_sentences, label, score, visualize=False):
    
    def post_or_comment_or_reply(index):
        for i, sentences in enumerate(all_sentences):
            if all_tokens[index] in sentences:
                if i == 0:
                    return 'post'
                elif i == 1:
                    return 'comment'
                else:
                    return 'reply'
    
    input_ids, ref_input_ids, attention_masks, sentence_counts = construct_input_ref_pair([post, comment, reply])
    
    one_hot_labels = torch.nn.functional.one_hot(torch.tensor(label), num_classes=len(labels))
    inputs = {'labels': one_hot_labels.type(torch.float).to(device),
          'input_ids1': input_ids[0].to(device),
          'input_ids2': input_ids[1].to(device),
          'input_ids3': input_ids[2].to(device),
          'attention_mask1': attention_masks[0].to(device),
          'attention_mask2': attention_masks[1].to(device),
          'attention_mask3': attention_masks[2].to(device),
          'sentence_count1': sentence_counts[0].to(device),
          'sentence_count2': sentence_counts[1].to(device),
          'sentence_count3': sentence_counts[2].to(device),
          'mode': 'triplet'
         }
    
    with torch.no_grad():
        embeddings = t_model(**inputs).hidden_states
        
    inputs = torch.stack(embeddings, dim=0)
    pred = forward_func_ig2(inputs, sentence_counts, t_model)
    # print(f'answer: {label}, predict: {torch.argmax(pred)}')
    
    result = []
    
    attribution, delta = ig.attribute(inputs=inputs, target=torch.argmax(pred), additional_forward_args=(sentence_counts, t_model), return_convergence_delta=True)
    attributions = summarize_attributions(attribution)
    f_attributions = torch.flatten(attributions)
    f_attributions = f_attributions[f_attributions.nonzero()].squeeze(1)
    abs_attributions = list(map(abs, map(float, f_attributions)))
    idx_attributions = []
    for j in range(len(abs_attributions)):
        idx_attributions.append((j, abs_attributions[j], f_attributions[j].item()))
    idx_attributions.sort(key=lambda x:x[1], reverse=True)
        
    top3 = idx_attributions[:3]
    
    if visualize:
        all_sentences = [['[[post]]'], post, ['[[comment]]'], comment, ['[[reply]]'], reply]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]
        
        vis_attributions = []
        
        j = 0
        for i in range(len(all_tokens)):
            if all_tokens[i] in ['[[post]]', '[[comment]]', '[[reply]]']:
                vis_attributions.append(0)
            else:
                vis_attributions.append(f_attributions[j].item())
                j += 1
                
        vis_attributions = torch.tensor(vis_attributions)
        
        score_vis = viz.VisualizationDataRecord(vis_attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                f'{label}, {score}',  # true label
                                                p_sentences + ' ' + c_sentences + ' ' + r_sentences,
                                                vis_attributions.sum(),
                                                all_tokens,
                                                delta)
        raw_text = ' '.join(post) + ' '.join(comment) + ' '.join(reply)
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])
        print(vis_attributions)
        
    else:
        where = []
        
        all_sentences = [post, comment, reply]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]

        for j in range(len(top3)):
            where.append(post_or_comment_or_reply(top3[j][0]))

        result.append([index, post, comment, reply, score, all_tokens[top3[0][0]], top3[0][2], where[0]])
        result.append([index, post, comment, reply, score, all_tokens[top3[1][0]], top3[1][2], where[1]])
        result.append([index, post, comment, reply, score, all_tokens[top3[2][0]], top3[2][2], where[2]])

        return result, label, torch.argmax(pred).item()

In [48]:
def splitbert_integrated_gradient_post_comment(index, post, comment, p_sentences, c_sentences, label, score, visualize=False):
    
    def post_or_comment_or_reply(index):
        for i, sentences in enumerate(all_sentences):
            if all_tokens[index] in sentences:
                if i == 0:
                    return 'post'
                elif i == 1:
                    return 'comment'
                else:
                    return 'reply'
    
    input_ids, ref_input_ids, attention_masks, sentence_counts = construct_input_ref_pair([post, comment])
    
    one_hot_labels = torch.nn.functional.one_hot(torch.tensor(label), num_classes=len(labels))
    inputs = {'labels': one_hot_labels.type(torch.float).to(device),
          'input_ids1': input_ids[0].to(device),
          'input_ids2': input_ids[1].to(device),
          'attention_mask1': attention_masks[0].to(device),
          'attention_mask2': attention_masks[1].to(device),
          'sentence_count1': sentence_counts[0].to(device),
          'sentence_count2': sentence_counts[1].to(device),
          'mode': 'post_comment'
         }
    
    with torch.no_grad():
        inputs = pc_model(**inputs).hidden_states
    inputs = inputs[0]
    # inputs = torch.stack(embeddings, dim=0)
    pred = forward_func_ig4(inputs, sentence_counts, pc_model)
    # print(f'answer: {label}, predict: {torch.argmax(pred)}')
    
    result = []
    
    attribution, delta = ig.attribute(inputs=inputs, target=torch.argmax(pred), additional_forward_args=(sentence_counts, pc_model), n_steps=1, return_convergence_delta=True)
    attributions = summarize_attributions(attribution)
    f_attributions = torch.flatten(attributions)
    f_attributions = f_attributions[f_attributions.nonzero()].squeeze(1)
    abs_attributions = list(map(abs, map(float, f_attributions)))
    idx_attributions = []
    for j in range(len(abs_attributions)):
        idx_attributions.append((j, abs_attributions[j], f_attributions[j].item()))
    idx_attributions.sort(key=lambda x:x[1], reverse=True)
        
    top3 = idx_attributions[:3]
    
    if visualize:
        all_sentences = [['[[post]]'], post, ['[[comment]]'], comment]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]
        
        vis_attributions = []
        
        j = 0
        for i in range(len(all_tokens)):
            if all_tokens[i] in ['[[post]]', '[[comment]]']:
                vis_attributions.append(0)
            else:
                vis_attributions.append(f_attributions[j].item())
                j += 1
                
        vis_attributions = torch.tensor(vis_attributions)
        
        score_vis = viz.VisualizationDataRecord(vis_attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                f'{label}, {score}',  # true label
                                                p_sentences + ' ' + c_sentences,
                                                vis_attributions.sum(),
                                                all_tokens,
                                                delta)
        raw_text = ' '.join(post) + ' '.join(comment)
        
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])
        print(vis_attributions)
        
    else:
        where = []
        
        all_sentences = [post, comment]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]

        for j in range(len(top3)):
            where.append(post_or_comment_or_reply(top3[j][0]))
            result.append([index, post, comment, score, all_tokens[top3[j][0]], top3[j][2], post_or_comment_or_reply(top3[j][0])])

        # result.append([index, post, comment, score, all_tokens[top3[0][0]], top3[0][2], where[0]])
        # result.append([index, post, comment, score, all_tokens[top3[1][0]], top3[1][2], where[1]])
        # result.append([index, post, comment, score, all_tokens[top3[2][0]], top3[2][2], where[2]])

        return result, label, torch.argmax(pred).item()

In [9]:
def get_indexes(filename):
    index_df = pd.read_csv(filename, encoding='UTF-8')
    index_df.columns = ['Unnamed: 0', 'prediction', 'label', 'score', 'idx']
    val_index = sorted(list(index_df.idx.values))
    
    return val_index

In [17]:
val_index = get_indexes(f'../predicting-satisfaction-using-graphs/csv/splitbert_classifier/post_comment/seg_seg/epoch_4_result.csv')

In [18]:
print(len(val_index))

100


In [15]:
def triplet_main(index_filename):
    index_list = get_indexes(index_filename)
    result_list = []
    label_pred_list = []
    
    for i in index_list:
        result, label, pred = splitbert_integrated_gradient_triplet(i, post_sequences[i], comment_sequences[i], reply_sequences[i], post_contents[i], comment_bodies[i], reply_contents[i], satisfactions[i], satisfactions_float[i])
        result_list.extend(result)
        
        for i in range(3):
            label_pred_list.append((label, pred))
        
    return result_list, label_pred_list   

In [16]:
def post_comment_main(index_filename):
    index_list = get_indexes(index_filename)
    result_list = []
    label_pred_list = []
    
    for i in index_list:
        result, label, pred = splitbert_integrated_gradient_post_comment(i, post_sequences[i], comment_sequences[i], post_contents[i], comment_bodies[i], satisfactions[i], satisfactions_float[i])
        result_list.extend(result)
        
        for i in range(len(result)):
            label_pred_list.append((label, pred))
        
    return result_list, label_pred_list   

In [52]:
if __name__ == "__main__":
    filtered = True
    
    # result_list, label_pred_list = main(val_index)
    result_list, label_pred_list = post_comment_main(val_index)
    print(len(result_list), len(label_pred_list))
    
    if filtered:
        filtered_list = []
        filtered_lp_list = []
        for result, lp in zip(result_list, label_pred_list):
            if abs(result[5]) >= 0.5:
                filtered_list.append(result)
                filtered_lp_list.append(lp)
    
    path = '../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/post_comment/'
    columns = ['idx', 'post_text', 'comment_text', 'score', 'attr_sentence', 'attr_score', 'origin']
    
    def make_csv(target_list, label_pred_list):
        for i in range(3):
            for j in range(3):
                result = []
                for k in range(len(target_list)):
                    if label_pred_list[k] == (i, j):
                        result.append(target_list[k])
                result_df = pd.DataFrame(result, columns=columns)
                
                if filtered:
                    filename = f'label_{i}_pred_{j}_attribution_filtered.csv'
                else:
                    filename = f'label_{i}_pred_{j}_attribution.csv'
                
                result_df.to_csv(path + filename)
    
    
    if not filtered:
        make_csv(result_list, label_pred_list)
    else:
        make_csv(filtered_list, filtered_lp_list)

271 271


In [18]:
if __name__ == "__main__":
    filtered = True
    
    # result_list, label_pred_list = main(val_index)
    result_list, label_pred_list = triplet_main(f'../predicting-satisfaction-using-graphs/csv/splitbert_classifier/triplet/seg_seg_snt/epoch_7_result.csv')
    print(len(result_list), len(label_pred_list))
    
    if filtered:
        filtered_list = []
        filtered_lp_list = []
        for result, lp in zip(result_list, label_pred_list):
            if abs(result[6]) >= 0.5:
                filtered_list.append(result)
                filtered_lp_list.append(lp)
    
    path = '../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/triplet/'
    columns = ['idx', 'post_text', 'comment_text', 'reply_text', 'score', 'attr_sentence', 'attr_score', 'origin']
    
    def make_csv(target_list, label_pred_list):
        for i in range(3):
            for j in range(3):
                result = []
                for k in range(len(target_list)):
                    if label_pred_list[k] == (i, j):
                        result.append(target_list[k])
                result_df = pd.DataFrame(result, columns=columns)
                
                if filtered:
                    filename = f'label_{i}_pred_{j}_attribution_filtered.csv'
                else:
                    filename = f'label_{i}_pred_{j}_attribution.csv'
                
                result_df.to_csv(path + filename)
    
    
    if not filtered:
        make_csv(result_list, label_pred_list)
    else:
        make_csv(filtered_list, filtered_lp_list)

300 300


In [53]:
index_list = []
target = 'post_comment'

for i in range(3):
    for j in range(3):
        print(f'label: {i}, pred: {j}')
        
        result_df = pd.read_csv(f'../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/{target}/label_{i}_pred_{j}_attribution_filtered.csv', encoding='UTF-8')
        print(result_df.origin.value_counts())
        
        if i == j:
            index_list.append(set(list(result_df.idx.values)))
        
        print()

label: 0, pred: 0
comment    10
post        7
Name: origin, dtype: int64

label: 0, pred: 1
comment    3
post       1
Name: origin, dtype: int64

label: 0, pred: 2
comment    3
post       2
Name: origin, dtype: int64

label: 1, pred: 0
post       12
comment     7
Name: origin, dtype: int64

label: 1, pred: 1
comment    14
post        8
Name: origin, dtype: int64

label: 1, pred: 2
comment    15
post        9
Name: origin, dtype: int64

label: 2, pred: 0
comment    5
post       4
Name: origin, dtype: int64

label: 2, pred: 1
comment    10
post        4
Name: origin, dtype: int64

label: 2, pred: 2
comment    15
post        7
Name: origin, dtype: int64



In [11]:
index_list = get_indexes(f'../predicting-satisfaction-using-graphs/csv/splitbert_classifier/post_comment/seg_seg/epoch_4_result.csv')

In [54]:
for i in index_list[:10]:
    # splitbert_integrated_gradient_triplet(i, post_sequences[i], comment_sequences[i], reply_sequences[i], post_contents[i], comment_bodies[i], reply_contents[i], satisfactions[i], satisfactions_float[i], True)
    splitbert_integrated_gradient_post_comment(i, post_sequences[i], comment_sequences[i], post_contents[i], comment_bodies[i], satisfactions[i], satisfactions_float[i], True)

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"0, 2.45",1 (1.00),"Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first",-0.72,"[[post]] Im the guy no one suspects is as messed up as I am....Am wealthy, have nice wife, kids, everyone envious of our home (8000 sf so not small in wealthiest county in the USA).....good looking and well you name it ..........But Im now unemployed, wife has and I think still sleeps with other men, am now drinking a lot, no sex in years, kids make jokes about my drinking to wife......I embarass my wife out in piblic, I dont work out anymore and have 50 lbs more than I did a yr and half ago, I am going broke and well Im a loser I think........But when i drink at least the pain and how much of a loser I am doesnt keep me awake all night and makes me feel like I can do stuff when drunk...... [[comment]] As bad as I feel about the depression, and I am pretty sure you are just taking out your pent up anger on him, you came out swinging first by calling him a moron over a little mistsle first"


tensor([ 0.0000, -0.9681,  0.0000,  0.2505])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"0, 2.45",0 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. Donât do these things please. If other people donât believe how hurt you are emotionally, try your best to let it not affect you.",0.71,"[[post]] I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally. I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. [[comment]] Donât do these things please. If other people donât believe how hurt you are emotionally, try your best to let it not affect you."


tensor([ 0.0000,  0.9666,  0.0000, -0.2564])
 Visualization For Score 


...

tensor([ 0.0000, -0.1633,  0.0000, -0.9761,  0.1433])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"2, 6.15",2 (1.00),"Soooo, i will keep it short, last 2 years have been hell for me, only thing keeping me afloat was my 'secret' youtube channel that only my friend knew about, until my other ""friend"" found out and deleted it from my computer, that was pretty bad for me, but luckily i found a gf and she kept me happy? Or well, at least kept me ""on line"", anyways now, 6 months ago she broke up with me, as u can imagine, first heart break, it hurt as hell, 3 days purely layin on bed, doing nothing, my friend showed up and told my parents about my channel, so later that day, my parents came to my room and asked why i didn't start again, the thing was it was waaay too frustrating to edit the videos with my old pc, so they offered to buy me 1 if i paid for it in parts, and now as i hit 1k subs, i am actually feeling getting better, socially, i do not hate my voice any more (my last channel i didn't use a mic) and in general i feel like i can talk to people now, so, from bottom of my heart, thank you mom and dad! This post was so cute haha, your friend and parents sound like cool people.",-1.20,"[[post]] Soooo, i will keep it short, last 2 years have been hell for me, only thing keeping me afloat was my 'secret' youtube channel that only my friend knew about, until my other ""friend"" found out and deleted it from my computer, that was pretty bad for me, but luckily i found a gf and she kept me happy? Or well, at least kept me ""on line"", anyways now, 6 months ago she broke up with me, as u can imagine, first heart break, it hurt as hell, 3 days purely layin on bed, doing nothing, my friend showed up and told my parents about my channel, so later that day, my parents came to my room and asked why i didn't start again, the thing was it was waaay too frustrating to edit the videos with my old pc, so they offered to buy me 1 if i paid for it in parts, and now as i hit 1k subs, i am actually feeling getting better, socially, i do not hate my voice any more (my last channel i didn't use a mic) and in general i feel like i can talk to people now, so, from bottom of my heart, thank you mom and dad! [[comment]] This post was so cute haha, your friend and parents sound like cool people."


tensor([ 0.0000, -0.9742,  0.0000, -0.2255])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
"1, 4.15",2 (1.00),hi iâm 19 and ive been diagnosed with depression for like two years. until i started medication ( around 4 months ago zoloft/sertraline)i would feel sad and cry all the time but now i donât ??? i only ever cry if i talk about things out loud up. up until i talk about it i donât feel sad. maybe it made me more numb iâm not sure. . maybe i suppress things? idk can anyone shed some light and if youâve experienced this pls help me out x Going to a therapist and talking about my problems and life really helps me destress throughout the week. Also you should try group therapy when I had to make the appointment I was tearing and shaking and then was shaking going to the first meeting. Then after the first meeting I started to enjoy it and it became like nothing and looked forward to going. The group I went to was mostly coping skills but Iâm gunna try a different kind of group next time.,-0.53,[[post]] hi iâm 19 and ive been diagnosed with depression for like two years. until i started medication ( around 4 months ago zoloft/sertraline)i would feel sad and cry all the time but now i donât ??? i only ever cry if i talk about things out loud up. up until i talk about it i donât feel sad. maybe it made me more numb iâm not sure. . maybe i suppress things? idk can anyone shed some light and if youâve experienced this pls help me out x [[comment]] Going to a therapist and talking about my problems and life really helps me destress throughout the week. Also you should try group therapy when I had to make the appointment I was tearing and shaking and then was shaking going to the first meeting. Then after the first meeting I started to enjoy it and it became like nothing and looked forward to going. The group I went to was mostly coping skills but Iâm gunna try a different kind of group next time.


tensor([ 0.0000,  0.3660, -0.8949, -0.2029,  0.0000,  0.0656,  0.1403])
